In [1]:
import json
from typing import List, Dict, Literal
from openai import OpenAI
from openai.types.chat.chat_completion_message import ChatCompletionMessage
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
client = OpenAI(base_url="https://openai.vocareum.com/v1")

In [5]:
class Memory:
  def __init__(self):
    self._messages: List[Dict[str, str]] = []

  def add_message(self, 
                  role: Literal["user", "system", "assistant"], 
                  content: str,
                  tool_calls: dict=dict(),
                  tool_call_id=None)->None:

    message = {
      "role": role,
      "content": content,
      "tool_calls": tool_calls
    }
    
    if role == "tool":
      message = {
        "role": role,
        "content": content,
        "tool_call_id": tool_call_id
      }
    
    self._messages.append(message)
  
  
  def get_messages(self) -> List[Dict[str, str]]:
    return self._messages
  
  
  def last_message(self) -> Dict[str, str]:
    if self._messages:
      return self._messages[-1]


  def reset(self) -> None:
    self._messages = []


In [6]:
memory = Memory()
memory.add_message(role="system", content="You're a helpful assistant")

In [7]:
memory.get_messages()

[{'role': 'system', 'content': "You're a helpful assistant", 'tool_calls': {}}]

In [9]:
memory.reset()
memory.get_messages()

[]

In [10]:
def chat_with_tools(user_question: str=None,
                    memory:Memory=None,
                    model: str="gpt-4o-mini",
                    temperature=0.0,
                    tools=None) -> str:
  
  messages = [{"role": "user", "content": user_question}]
  if memory:
    if user_question:
      memory.add_message(role="user", content=user_question)
    
    messages = memory.get_messages()
  
  response = client.chat.completions.create(
    model=model,
    temperature=temperature,
    messages=messages,
    tools=tools,
  )
  
  ai_message = str(response.choices[0].message.content)
  tool_calls = response.choices[0].message.tool_calls
  
  if memory:
    memory.add_message(role="assistant", content=ai_message, tool_calls=tool_calls)
  
  return ai_message


In [12]:
chat_with_tools(
  "2 to the power of -5?",
  model="gpt-3.5-turbo",
)

'1/32'

In [16]:
def power(base: float, exponent: float):
  """Exponentatiation: base to the power of exponent"""
  
  return base ** exponent

In [19]:
tools = [{
  "type": "function",
  "function": {
    "name": "power",
    "description": "Exponentatiation: base to the power of exponent",
    "parameters": {
      "type": "object",
      "properties": {
        "base": {"type": "number"},
        "exponent": {"type": "number"}
      },
      "required": ["base", "exponent"],
      "additionalProperties": False
    },
    "strict": True
  }
}]

In [21]:
memory = Memory()
memory.add_message(role="system", content="You're a helpful assistant")

In [22]:
memory.get_messages()

[{'role': 'system', 'content': "You're a helpful assistant", 'tool_calls': {}}]

In [23]:
chat_with_tools(
  "2 to the power of -5?",
  model="gpt-3.5-turbo",
  tools=tools,
  memory=memory
)

'None'

In [24]:
memory.get_messages()

[{'role': 'system', 'content': "You're a helpful assistant", 'tool_calls': {}},
 {'role': 'user', 'content': '2 to the power of -5?', 'tool_calls': {}},
 {'role': 'assistant',
  'content': 'None',
  'tool_calls': [ChatCompletionMessageToolCall(id='call_DdLAn9KTn8EdPAKilBgit7pO', function=Function(arguments='{"base":2,"exponent":-5}', name='power'), type='function')]}]

In [26]:
memory.last_message()['tool_calls']

[ChatCompletionMessageToolCall(id='call_DdLAn9KTn8EdPAKilBgit7pO', function=Function(arguments='{"base":2,"exponent":-5}', name='power'), type='function')]

In [27]:
tool_call_id = memory.last_message()['tool_calls'][0].id
tool_call_id

'call_DdLAn9KTn8EdPAKilBgit7pO'

In [28]:
args = json.loads(memory.last_message()['tool_calls'][0].function.arguments)
args

{'base': 2, 'exponent': -5}

In [29]:
result = power(args["base"], args["exponent"])
result

0.03125

In [30]:
memory.add_message(role="tool", content=str(result), tool_call_id=tool_call_id)

In [31]:
memory.get_messages()

[{'role': 'system', 'content': "You're a helpful assistant", 'tool_calls': {}},
 {'role': 'user', 'content': '2 to the power of -5?', 'tool_calls': {}},
 {'role': 'assistant',
  'content': 'None',
  'tool_calls': [ChatCompletionMessageToolCall(id='call_DdLAn9KTn8EdPAKilBgit7pO', function=Function(arguments='{"base":2,"exponent":-5}', name='power'), type='function')]},
 {'role': 'tool',
  'content': '0.03125',
  'tool_call_id': 'call_DdLAn9KTn8EdPAKilBgit7pO'}]

In [32]:
ai_message = chat_with_tools(
  model="gpt-3.5-turbo",
  memory=memory
)

In [33]:
memory.get_messages()

[{'role': 'system', 'content': "You're a helpful assistant", 'tool_calls': {}},
 {'role': 'user', 'content': '2 to the power of -5?', 'tool_calls': {}},
 {'role': 'assistant',
  'content': 'None',
  'tool_calls': [ChatCompletionMessageToolCall(id='call_DdLAn9KTn8EdPAKilBgit7pO', function=Function(arguments='{"base":2,"exponent":-5}', name='power'), type='function')]},
 {'role': 'tool',
  'content': '0.03125',
  'tool_call_id': 'call_DdLAn9KTn8EdPAKilBgit7pO'},
 {'role': 'assistant',
  'content': '2 to the power of -5 is equal to 0.03125.',
  'tool_calls': None}]